In [63]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from glob import glob
from tensorflow.python.ops import nn

In [95]:
import math
import h5py

def get_total_num_examples(hdf5_file_list):
    '''
    Quickly extracts total examples represented in an hdf5 file list. Can 
    be used to calculate total steps to take (when 1 step represents going 
    through a batch of examples)
    '''
    
    num_examples = 0
    for filename in hdf5_file_list:
        with h5py.File(filename,'r') as hf:
            num_examples += hf['features'].shape[0]

    return num_examples


def get_fan_in(tensor, type='NHWC'):
    '''
    Get the fan in (number of in channels)
    '''

    return int(tensor.get_shape()[-1])

def maxnorm(norm_val=7):
    '''
    Torch7 style maxnorm. After gradients are applied in a training step,
    the weights of each layer are clipped. This maxnorm applies to 
    convolutional layers as well as fully connected layers and does 
    not apply to biases.
    '''

    weights = [v for v in tf.all_variables()
               if ('weights' in v.name)]

    for weight in weights:
        op_name = '{}/maxnorm'.format(weight.name.split('/weights')[0])
        maxnorm_update = weight.assign(
            tf.clip_by_norm(weight, norm_val, axes=[0], name=op_name))
        tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                             maxnorm_update)
        
    return None

In [96]:
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import random_ops

def torch_initializer(stdv, dtype=dtypes.float32):
    '''
    Torch7 style initializer. Can be used for weights and biases.
    '''
    def _initializer(shape, dtype=dtype, partition_info=None, seed=1337):
        return random_ops.random_uniform(shape, -stdv, stdv, dtype, seed=seed)
    return _initializer


def torch_conv_initializer(filter_shape, fan_in, dtype=dtypes.float32):
    '''
    Wraps the calculation of the standard dev for convolutional layers:
    stdv = 1 / sqrt( filter_width * filter_height * fan_in )
    '''
    stdv = 1. / math.sqrt(filter_shape[0] * filter_shape[1] * fan_in)
    return torch_initializer(stdv)


def torch_fullyconnected_initializer(fan_in, dtype=dtypes.float32):
    '''
    Wraps the calculation of the standard dev for fully connected layers:
    stdv = 1 / sqrt( fan_in )
    '''
    stdv = 1. / math.sqrt(fan_in)
    return torch_initializer(stdv)

In [97]:
def basset(features, labels, is_training=True):
    '''
    Basset - Kelley et al Genome Research 2016
    '''

    with slim.arg_scope([slim.conv2d], padding='VALID',
                        activation_fn=None):

        # Layer 1: conv layer to batch norm to relu to max pool.
        conv1_filter_size = [1, 19]
        net = slim.conv2d(
            features, 300, conv1_filter_size,
            weights_initializer=torch_conv_initializer(
                conv1_filter_size, get_fan_in(features)),
            biases_initializer=torch_conv_initializer(
                conv1_filter_size, get_fan_in(features)),
            scope='conv1/conv')
        net = slim.batch_norm(net, center=True, scale=True,
                              activation_fn=nn.relu, 
                              is_training=is_training,
                              scope='conv1/batchnorm')
        net = slim.max_pool2d(net, [1, 3], stride=[1, 3], 
            scope='conv1/maxpool')

        # Layer 2: conv layer to batch norm to relu to max pool.
        conv2_filter_size = [1, 11]
        net = slim.conv2d(
            net, 200, conv2_filter_size,
            weights_initializer=torch_conv_initializer(
                conv2_filter_size, get_fan_in(net)),
            biases_initializer=torch_conv_initializer(
                conv2_filter_size, get_fan_in(net)),
            scope='conv2/conv')
        net = slim.batch_norm(net, center=True, scale=True, 
                              activation_fn=nn.relu, 
                              is_training=is_training,
                              scope='conv2/batchnorm')
        net = slim.max_pool2d(net, [1, 4], stride=[1, 4], 
            scope='conv2/maxpool')

    net = slim.flatten(net, scope='flatten')

    with slim.arg_scope([slim.fully_connected], activation_fn=None):

        # Layer 4: fully connected layer to relu to dropout
        net = slim.fully_connected(
            net, 1000, 
            weights_initializer=torch_fullyconnected_initializer(
                get_fan_in(net)),
            biases_initializer=torch_fullyconnected_initializer(
                get_fan_in(net)),
            scope='fullyconnected1/fullyconnected')
        net = slim.batch_norm(net, center=True, scale=True,
                              activation_fn=nn.relu, 
                              is_training=is_training,
                              scope='fullyconnected1/batchnorm')
        net = slim.dropout(net, keep_prob=0.7, is_training=is_training,
            scope='fullyconnected1/dropout')

        # Layer 5: fully connected layer to relu to dropout
        net = slim.fully_connected(
            net, 1000, 
            weights_initializer=torch_fullyconnected_initializer(
                get_fan_in(net)),
            biases_initializer=torch_fullyconnected_initializer(
                get_fan_in(net)),
            scope='fullyconnected2/fullyconnected')
        net = slim.batch_norm(net, center=True, scale=True,
                              activation_fn=nn.relu, 
                              is_training=is_training,
                              scope='fullyconnected2/batchnorm')
        net = slim.dropout(net, keep_prob=0.7, is_training=is_training,
            scope='fullyconnected2/dropout')

        # OUT
        logits = slim.fully_connected(
            net, int(labels.get_shape()[-1]), activation_fn=None,
            weights_initializer=torch_fullyconnected_initializer(
                get_fan_in(net)),
            biases_initializer=torch_fullyconnected_initializer(
                get_fan_in(net)),
            scope='out')

    # Torch7 style maxnorm
    maxnorm(norm_val=7)

    return logits

In [98]:
def train(X, y,
          model_builder,
          loss_fn,
          optimizer_fn,
          optimizer_params,
          restore,
          stopping_criterion,
          batch_size,
          OUT_DIR,
          global_step_val):
    '''
    Wraps the routines needed for tf-slim
    '''

    with tf.Graph().as_default() as g:
        
        features, labels, metadata = 

        # model
        predictions = model_builder(features, labels, is_training=True)

        # loss
        total_loss = loss_fn(predictions, labels)

        # optimizer
        optimizer = optimizer_fn(**optimizer_params)

        # train op
        train_op = slim.learning.create_train_op(total_loss, optimizer)

        # build metrics
        summary_op = metrics_fn(total_loss, predictions, labels)

        if restore:
            checkpoint_path = tf.train.latest_checkpoint(OUT_DIR)
            variables_to_restore = slim.get_model_variables()
            variables_to_restore.append(slim.get_global_step()) 
            init_assign_op, init_feed_dict = slim.assign_from_checkpoint(
                checkpoint_path,
                variables_to_restore)
            
            # create init assignment function
            def InitAssignFn(sess):
                sess.run(init_assign_op, init_feed_dict)
                
            slim.learning.train(train_op,
                                OUT_DIR,
                                init_fn=InitAssignFn,
                                number_of_steps=global_step_val,
                                summary_op=summary_op,
                                save_summaries_secs=20)

        else:
            slim.learning.train(train_op,
                                OUT_DIR,
                                number_of_steps=global_step_val,
                                summary_op=summary_op,
                                save_summaries_secs=20)

    return None

SyntaxError: invalid syntax (<ipython-input-98-36c17e947115>, line 17)

In [99]:
import numpy as np

def setup_queue(features, labels, metadata, capacity=10000):
    '''
    Set up data queue as well as queue runner. The shapes of the
    tensors are inferred from the inputs, so input shapes must be
    set before this function is called.
    '''

    with tf.variable_scope('datalayer'):
        queue = tf.FIFOQueue(capacity,
                             [tf.float32, tf.float32, tf.string],
                             shapes=[features.get_shape()[1:],
                                     labels.get_shape()[1:],
                                     metadata.get_shape()[1:]])
        enqueue_op = queue.enqueue_many([features, labels, metadata])
        queue_runner = tf.train.QueueRunner(
            queue=queue,
            enqueue_ops=[enqueue_op],
            close_op=queue.close(),
            cancel_op=queue.close(cancel_pending_enqueues=True))
        tf.train.add_queue_runner(queue_runner, tf.GraphKeys.QUEUE_RUNNERS)

    return queue


def get_hdf5_list_reader_pyfunc(hdf5_files, batch_size):
    '''
    Takes in a list of hdf5 files and generates a tensorflow op that returns a 
    group of examples and labels when called in the graph. Be aware that this
    setup uses global variables that must be initialized first to make this
    work.
    '''

    # Get all file handles before starting learning.
    h5py_handles = [ h5py.File(filename) for filename in hdf5_files ]

    # Check shapes from the hdf5 file so that we can set the tensor shapes
    feature_shape = h5py_handles[0]['features'].shape[1:]
    label_shape = h5py_handles[0]['labels'].shape[1:]

    def hdf5_reader_fn():
        '''
        Given batch start and stop, pulls those examples from hdf5 file
        '''
        global batch_start
        global batch_end
        global filename_index

        # check if at end of file, and move on to the next file
        if batch_end > h5py_handles[filename_index]['features'].shape[0]:
            print hdf5_files[filename_index]
            filename_index += 1
            batch_start = 0
            batch_end = batch_size

        if filename_index >= len(h5py_handles):
            filename_index = 0
            batch_start = 0
            batch_end = batch_size

        current_handle = h5py_handles[filename_index]

        features = current_handle['features'][batch_start:batch_end,:,:,:]
        labels = current_handle['labels'][batch_start:batch_end,:]
        metadata = current_handle['regions'][batch_start:batch_end].reshape(
            (batch_size, 1))

        batch_start += batch_size
        batch_end += batch_size

        return [features, labels, metadata]

    [py_func_features, py_func_labels, py_func_metadata] = tf.py_func(
        hdf5_reader_fn,
        [],
        [tf.float32, tf.float32, tf.string],
        stateful=True)

    # Set the shape so that we can infer sizes etc in later layers.
    py_func_features.set_shape([batch_size,
                                feature_shape[0],
                                feature_shape[1],
                                feature_shape[2]])
    py_func_labels.set_shape([batch_size, label_shape[0]])
    py_func_metadata.set_shape([batch_size, 1])
    
    return py_func_features, py_func_labels, py_func_metadata


def load_data_from_filename_list(hdf5_files, batch_size):
    '''
    Put it all together
    '''

    global batch_start
    global batch_end
    global filename_index

    batch_start = 0
    batch_end = batch_size
    filename_index = 0

    [hdf5_features, hdf5_labels, hdf5_metadata] = get_hdf5_list_reader_pyfunc(hdf5_files,
                                                                              batch_size)

    queue = setup_queue(hdf5_features, hdf5_labels, hdf5_metadata)

    [features, labels, metadata] = queue.dequeue_many(batch_size)

    return features, labels, metadata

In [100]:
def main(X_train, X_valid, y_train, y_valid):
    # TODO fix input of info to make easier to run
    OUT_DIR = './log'
    batch_size = 128
    epochs = 4

    # This all needs to be cleaned up into some kind of init function...
    num_train_examples = 1000 #y_train.get_shape()[0]
    train_steps = num_train_examples / batch_size - 100
    print train_steps
    train_steps = 100 # for now. just to make it easier to test
    print 'Num train examples: {}'.format(num_train_examples)

    num_valid_examples = 50 #y_valid.get_shape()[0]
    valid_steps = num_valid_examples / batch_size - 100
    print 'Num valid examples: {}'.format(num_valid_examples)

    # Should epoch level be where things are exposed here? or is this loop abstractable too?
    for epoch in xrange(epochs):
        print "EPOCH:", str(epoch)

        restore = bool(epoch)

        # Run training
        train(X_train, y_train,
              basset,
              slim.losses.sigmoid_cross_entropy,
            tf.train.RMSPropOptimizer,
            {'learning_rate': 0.002, 'decay':0.98, 'momentum':0.0, 'epsilon':1e-8},
            restore,
            'Not yet implemented',
            batch_size,
            '{}/train'.format(OUT_DIR),
            (epoch+1)*train_steps)

        # Get last checkpoint
        checkpoint_path = tf.train.latest_checkpoint('{}/train'.format(OUT_DIR))

In [101]:
X_train = tf.ones((1000, 4, 100, 1), dtype='float32')
X_valid = np.ones((50, 4, 100, 1), dtype='float32')
y_train = tf.ones((1000,1), dtype='float32')
y_valid = np.ones((50,1), dtype='float32')
main(X_train, X_valid, y_train, y_valid)

-93
Num train examples: 1000
Num valid examples: 50
EPOCH: 0


ValueError: Variable conv1/conv/weights already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/home/deeperswag/.conda/envs/tf-model/lib/python2.7/site-packages/tensorflow/contrib/framework/python/ops/variables.py", line 230, in variable
    caching_device=caching_device)
  File "/home/deeperswag/.conda/envs/tf-model/lib/python2.7/site-packages/tensorflow/contrib/framework/python/ops/arg_scope.py", line 171, in func_with_args
    return func(*args, **current_args)
  File "/home/deeperswag/.conda/envs/tf-model/lib/python2.7/site-packages/tensorflow/contrib/framework/python/ops/variables.py", line 266, in model_variable
    caching_device=caching_device, device=device)
